In [20]:
import requests
import pandas as pd
import io
import datetime
from IPython.display import JSON

In [21]:
csv = requests.get("https://opendata.arcgis.com/datasets/37abda537d17458bae6677b8ab75fcb9_0.csv").content
main_df = pd.read_csv(io.StringIO(csv.decode('utf-8')))

In [25]:
main_df

,County,Age,Age_group,Gender,Jurisdiction,Travel_related,Origin,EDvisit,Hospitalized,Died,Case_,Contact,Case1,EventDate,ChartDate,ObjectId
0,Palm Beach,99.0,85+ years,Male,FL resident,No,NaN,NaN,UNKNOWN,NaN,Yes,UNKNOWN,2020/05/14 05:00:00+00,2020/04/29 00:00:00+00,2020/05/14 05:00:00+00,1
1,Broward,21.0,15-24 years,Female,FL resident,No,NaN,NO,NO,NaN,Yes,Yes,2020/05/14 05:00:00+00,2020/05/08 00:00:00+00,2020/05/14 05:00:00+00,2
2,Palm Beach,86.0,85+ years,Female,FL resident,No,NaN,UNKNOWN,UNKNOWN,NaN,Yes,NaN,2020/05/14 05:00:00+00,2020/05/14 00:00:00+00,2020/05/14 05:00:00+00,3
3,Dade,35.0,35-44 years,Female,FL resident,No,NaN,UNKNOWN,UNKNOWN,NaN,Yes,NaN,2020/05/14 05:00:00+00,2020/05/14 00:00:00+00,2020/05/14 05:00:00+00,4
4,Dade,47.0,45-54 years,Female,FL resident,No,NaN,UNKNOWN,UNKNOWN,NaN,Yes,NaN,2020/05/14 05:00:00+00,2020/05/14 00:00:00+00,2020/05/14 05:00:00+00,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291624,Seminole,17.0,15-24 years,Male,FL resident,Unknown,NaN,UNKNOWN,UNKNOWN,NaN,Yes,NaN,2020/07/13 05:00:00+00,2020/07/13 07:40:26+00,2020/07/13 05:00:00+00,291625
291625,Dade,72.0,65-74 years,Male,FL resident,Unknown,NaN,YES,UNKNOWN,NaN,Yes,NaN,2020/07/13 05:00:00+00,2020/07/02 00:00:00+00,2020/07/13 05:00:00+00,291626
291626,Duval,51.0,45-54 years,Male,FL resident,Unknown,NaN,UNKNOWN,UNKNOWN,NaN,Yes,NaN,2020/07/13 05:00:00+00,2020/07/13 08:13:51+00,2020/07/13 05:00:00+00,291627
291627,Duval,18.0,15-24 years,Female,FL resident,Unknown,NaN,UNKNOWN,UNKNOWN,NaN,Yes,NaN,2020/07/13 05:00:00+00,2020/07/13 08:13:51+00,2020/07/13 05:00:00+00,291628


In [23]:
filt = main_df["Hospitalized"] == "YES"
df = main_df[filt]

In [24]:
df

,County,Age,Age_group,Gender,Jurisdiction,Travel_related,Origin,EDvisit,Hospitalized,Died,Case_,Contact,Case1,EventDate,ChartDate,ObjectId
7,Hillsborough,79.0,75-84 years,Male,FL resident,No,NaN,NO,YES,NaN,Yes,Yes,2020/06/23 05:00:00+00,2020/06/16 00:00:00+00,2020/06/23 05:00:00+00,8
11,Lee,56.0,55-64 years,Female,FL resident,No,NaN,YES,YES,NaN,Yes,Yes,2020/05/15 05:00:00+00,2020/05/10 00:00:00+00,2020/05/15 05:00:00+00,12
13,Dade,41.0,35-44 years,Male,FL resident,Unknown,NaN,YES,YES,NaN,Yes,UNKNOWN,2020/05/15 05:00:00+00,2020/05/15 00:00:00+00,2020/05/15 05:00:00+00,14
14,Dade,53.0,45-54 years,Female,FL resident,No,NaN,YES,YES,NaN,Yes,Yes,2020/05/15 05:00:00+00,2020/05/14 00:00:00+00,2020/05/15 05:00:00+00,15
15,Polk,86.0,85+ years,Male,FL resident,No,NaN,YES,YES,NaN,Yes,NO,2020/05/15 05:00:00+00,2020/05/14 00:00:00+00,2020/05/15 05:00:00+00,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289925,Brevard,73.0,65-74 years,,FL resident,No,NaN,YES,YES,NaN,Yes,UNKNOWN,2020/07/13 05:00:00+00,2020/07/13 00:00:00+00,2020/07/13 05:00:00+00,289926
290085,Brevard,59.0,55-64 years,Male,FL resident,No,NaN,YES,YES,NaN,Yes,UNKNOWN,2020/07/13 05:00:00+00,2020/06/30 00:00:00+00,2020/07/13 05:00:00+00,290086
290401,Sarasota,73.0,65-74 years,Female,FL resident,No,NaN,YES,YES,NaN,Yes,UNKNOWN,2020/07/13 05:00:00+00,2020/07/04 00:00:00+00,2020/07/13 05:00:00+00,290402
290957,Collier,50.0,45-54 years,Male,FL resident,Unknown,NaN,YES,YES,NaN,Yes,UNKNOWN,2020/07/13 05:00:00+00,2020/07/04 00:00:00+00,2020/07/13 05:00:00+00,290958
